In [1]:
import os
import shutil
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
pd.set_option("display.max_colwidth",None)

In [3]:
from google.cloud import storage

In [4]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True) 

In [6]:
twitter_df=spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/kishorkumarreddy/filtered_data_for_analysis_1')
c = twitter_df

23/03/01 18:43:33 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
twitter_df.createOrReplaceTempView("twitter_df")

In [8]:
query = '''
select retweeted as is_retweeted,
retweeted_status.id as rt_original_id,
retweeted_status.user.screen_name as rt_original_user,
user.screen_name as handle_name,
user.verified as is_verified,
user.followers_count as followers,
user.statuses_count as total_tweets
from twitter_df'''
tw_sq1 = spark.sql(query)

In [9]:
twitter_df1 = tw_sq1.withColumn("tweet_type",when(col("rt_original_id").isNull() != True,"retweet").otherwise("tweet")).filter(col('tweet_type') == 'tweet')

In [10]:
twitter_df1.createOrReplaceTempView("twitter_df")

In [11]:
twitter_df1.printSchema()

root
 |-- is_retweeted: string (nullable = true)
 |-- rt_original_id: long (nullable = true)
 |-- rt_original_user: string (nullable = true)
 |-- handle_name: string (nullable = true)
 |-- is_verified: boolean (nullable = true)
 |-- followers: long (nullable = true)
 |-- total_tweets: long (nullable = true)
 |-- tweet_type: string (nullable = false)



# Based on Overall Content (Tweets), the Most Prolific Tweeters

In [39]:
spark.sql('SELECT handle_name as user, max(total_tweets) as total_content FROM twitter_df GROUP BY user ORDER BY total_content DESC LIMIT 10').toPandas()

,user,total_content
0,soldier_777,4327697
1,zazoomblog,4197074
2,PulpNews,4187315
3,missb62,3137543
4,sectest9,3080006
5,marekingu,2897601
6,Knewz_Currently,2850254
7,jornalistavitor,2815923
8,filafresh,2736894
9,paul_cude,2662698


# Based on Original Content (Tweets), the Most Prolific Tweets

In [13]:
spark.sql('SELECT handle_name as user, count(*) as original_content FROM twitter_df WHERE tweet_type = "tweet" GROUP BY user ORDER BY original_content DESC LIMIT 10').toPandas()

,user,original_content
0,ParentSecurity,3128
1,AJBlackston,1323
2,PulpNews,1290
3,getthatrightgtr,1147
4,AirLiveRadio,1038
5,Bgm117771,1002
6,MarchingTruth2,873
7,oodlu_tweets,858
8,itsrohitchouhan,857
9,DuoInspirations,837


In [14]:
twitter_df1 = tw_Q1.withColumn("tweet_type",when(col("rt_original_id").isNull() == True,"tweet"))
twitter_df1.createOrReplaceTempView("twitter_df")
twitter_df1.count()

1005167

Identifying the Most Prolific Authors

In [15]:
query = '''select handle_name, 
count(*) 
from twitter_df 
where tweet_type = "tweet" 
group by handle_name 
order by count(*) 
desc limit 10'''
spark.sql(query).toPandas()

,handle_name,count(1)
0,ParentSecurity,3128
1,AJBlackston,1323
2,PulpNews,1290
3,getthatrightgtr,1147
4,AirLiveRadio,1038
5,Bgm117771,1002
6,MarchingTruth2,873
7,oodlu_tweets,858
8,itsrohitchouhan,857
9,DuoInspirations,837


# Based on Retweets Count (Most Prolific Retweets)

In [16]:
spark.sql('select rt_original_user, count(*) from twitter_df where rt_original_user IS NOT NULL group by rt_original_user order by count(*) desc').toPandas()

,rt_original_user,count(1)
0,Golshifteh,7451
1,GretaThunberg,4504
2,PahlaviReza,3786
3,Hornystepsis1,3638
4,robbeorn,3277
...,...,...
80269,baddiejenny92,1
80270,rouse_dr,1
80271,CabellsMedicine,1
80272,mattitakubel,1


In [17]:
twitter_df.describe()

summary,created_at,favorite_count,filter_level,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,lang,quote_count,quoted_status_id,quoted_status_id_str,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,source,text,timestamp_ms,tweet_text,stripped
count,1005167,1005167,1005167,1005167,1005167,33709,27232,27232,33709,33709,1005167,1005167,72963,72963,72946,1005167,1005167,1005167,703935,1005167,1005167,1005167,1005167,1005167
mean,null,0.0,null,1.566841370743179...,1.566841370743179...,433.0,1.563454478121966...,1.563454478121966...,5.277987172089125...,5.277987172089125...,null,0.0,1.559602036301624...,1.559602036301624...,420.0,0.0,0.0,null,15417.625925925926,null,null,1.662399067946775...,null,null
stddev,null,0.0,null,3.115244689557993...,3.115244689557993...,null,4.269803723556284...,4.269803723556284...,6.307440523906169...,6.307440523906169...,null,0.0,5.343131196135945...,5.343131196135945...,null,0.0,0.0,null,543175.3414855197,null,null,7.427322124370399E9,null,null
min,Fri Apr 08 00:00:...,0,low,1511197436458459137,1511197436458459137,00Fuzz,1559129819,1018119192426491904,2651,1000060610518896640,en,0,124623659041161217,1003686491070128129,!!! Parents Alert...,0,0,,https,"<a href=""HTTP://b...",! #students #scho...,1649132520199,! #students #scho...,after a lon...
max,Wed Sep 28 23:59:...,0,low,1623307508390694913,1623307508390694913,zzzzzryu,1623288183676096512,993241772317913088,1622359362378375171,999988014032945152,en,0,1623294581193203714,999010989751259136,🫶~The ❤️ back to...,0,0,RT,… https,"<a href=""https://...",🫶🏽 Give back to...,1675861644252,🫶🏽 give back to...,🫶🏽 give back to...


In [18]:
df_retweetCounts = twitter_df.select([
    twitter_df.retweet_count.alias("direct_retweet_count"),
    twitter_df.quoted_status.retweet_count.alias("quoted_status.retweet_count"),
    twitter_df.retweeted_status.quoted_status.retweet_count.alias("retweeted_status.quoted_status.retweet_count"),
    twitter_df.retweeted_status.retweet_count.alias("retweeted_status.retweet_count"),
    twitter_df.retweeted_status.reply_count.alias("retweeted_status.reply_count"),
    twitter_df.reply_count.alias("reply_count"),
    twitter_df.tweet_text
]).limit(10000).toPandas()

In [19]:
df_retweetCounts.count()

direct_retweet_count                            10000
quoted_status.retweet_count                       535
retweeted_status.quoted_status.retweet_count      329
retweeted_status.retweet_count                   7935
retweeted_status.reply_count                     7935
reply_count                                     10000
tweet_text                                      10000
dtype: int64

In [20]:
df_retweetCounts.describe()

,direct_retweet_count,quoted_status.retweet_count,retweeted_status.quoted_status.retweet_count,retweeted_status.retweet_count,retweeted_status.reply_count,reply_count
count,10000.0,535.000000,329.000000,7935.000000,7935.000000,10000.0
mean,0.0,1073.067290,907.468085,1370.853938,232.817139,0.0
std,0.0,2663.100875,2459.589249,2528.899777,425.764715,0.0
min,0.0,0.000000,0.000000,1.000000,0.000000,0.0
25%,0.0,10.000000,21.000000,19.000000,0.000000,0.0
50%,0.0,106.000000,109.000000,184.000000,13.000000,0.0
75%,0.0,969.500000,969.000000,1390.500000,232.000000,0.0
max,0.0,22407.000000,20944.000000,14588.000000,2293.000000,0.0


In [21]:
df_original_tweets = twitter_df.filter("retweeted_status.retweet_count is not null") 
df_original_tweets.count()

652559

In [22]:
count1 = twitter_df.count()
count2 = twitter_df.filter('retweeted_status is null').count()
print('Count of all tweets:', count1)
print('Count of all original tweets:', count2)

Count of all tweets: 1005167
Count of all original tweets: 352608


## Identifying the Most Prolific (or) Influential Twitterers Using the Count of Tweets

In [23]:
print(twitter_df.columns)

['coordinates', 'created_at', 'display_text_range', 'entities', 'extended_entities', 'extended_tweet', 'favorite_count', 'favorited', 'filter_level', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'quote_count', 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink', 'quoted_text', 'reply_count', 'retweet_count', 'retweeted', 'retweeted_from', 'retweeted_status', 'source', 'text', 'timestamp_ms', 'truncated', 'tweet_text', 'user', 'withheld_in_countries', 'stripped']


### Extracting Important Columns and fields from the filtered dataframe

In [40]:
#tw_df.printSchema()

In [27]:
df_coords = twitter_df.select([
    twitter_df.coordinates.alias("direct_coordinates"),
    twitter_df.geo.coordinates.alias("geo.coordinates"),
    twitter_df.place.bounding_box.coordinates.alias("place.bounding_box.coordinates"),
    twitter_df.place.country_code.alias("place.country_code"),
    twitter_df.place.country.alias("place.country"),
    twitter_df.place['name'].alias("place.name"),
    twitter_df.place.full_name.alias("place.full_name"),
    twitter_df.user.location.alias("user.location"),
    twitter_df.tweet_text,
    twitter_df.user.withheld_in_countries
]).limit(30000).toPandas()

In [28]:
df_coords.describe()

,direct_coordinates,geo.coordinates,place.bounding_box.coordinates,place.country_code,place.country,place.name,place.full_name,user.location,tweet_text,user.withheld_in_countries
count,72,72,193,193,193,193,193,12523,30000,30000
unique,60,60,161,15,17,162,162,5560,14065,1
top,"([-71.4128343, 41.8239891], Point)","[41.8239891, -71.4128343]","[[[-71.474186, 41.772455], [-71.474186, 41.861713], [-71.369479, 41.861713], [-71.369479, 41.772455]]]",US,United States,Providence,"Providence, RI",United States,let go of students in #sharif_university #mahsaamini https://t.co/jjnszzjjf5,[]
freq,5,5,5,124,123,5,5,225,2563,30000


### Taking the non-null values from the field user.withheld_in_countries to avoid bad plotting

In [29]:
twitter_df.filter("user.withheld_in_countries is not NULL").count()

1005167

In [31]:
twitter_df.limit(5).toPandas().columns

Index(['coordinates', 'created_at', 'display_text_range', 'entities',
       'extended_entities', 'extended_tweet', 'favorite_count', 'favorited',
       'filter_level', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
       'lang', 'place', 'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'quoted_text', 'reply_count', 'retweet_count', 'retweeted',
       'retweeted_from', 'retweeted_status', 'source', 'text', 'timestamp_ms',
       'truncated', 'tweet_text', 'user', 'withheld_in_countries', 'stripped'],
      dtype='object')

In [32]:
df_coords_analysis_df = twitter_df.select([
    twitter_df.created_at,
    twitter_df.id,
    twitter_df.geo.coordinates.alias("geo_coordinates"),
    twitter_df.user['name'].alias("user_name"), 
    twitter_df.user.followers_count.alias("followers_count"), 
    twitter_df.user.verified.alias("verified_user"),
    twitter_df.user.location.alias("user_location"),
    twitter_df.user.description.alias("user_description"),
    twitter_df.retweeted_status.reply_count.alias("reply_count"),
    twitter_df.retweeted_status.retweet_count.alias("retweet_count"),
    twitter_df.retweeted_status.alias("retweeted_status"),
    twitter_df.tweet_text,
    twitter_df.text,
])itter

df_coords_analysis_df.limit(5).toPandas()

23/03/01 18:57:54 ERROR org.apache.spark.network.client.TransportResponseHandler: Still have 2 requests outstanding when connection from /10.128.0.68:40754 is closed
23/03/01 18:57:54 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: Error trying to remove broadcast 36 from block manager BlockManagerId(32, hub-msca-bdp-dphub-students-kishorkumarreddy-sw-qdhr.c.msca-bdp-students.internal, 40227, None)
java.io.IOException: Connection from /10.128.0.68:40754 closed
	at org.apache.spark.network.client.TransportResponseHandler.channelInactive(TransportResponseHandler.java:146)
	at org.apache.spark.network.server.TransportChannelHandler.channelInactive(TransportChannelHandler.java:117)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:262)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:248)
	at io.netty.channel.AbstractChannelHandlerContext.fireChannelInactive(Abstra

created_at                   id geo_coordinates  \
0  Sat Oct 29 18:36:18 +0000 2022  1586426715760459778            None   
1  Sat Oct 29 18:36:18 +0000 2022  1586426719090589696            None   
2  Sat Oct 29 18:36:55 +0000 2022  1586426871524343814            None   
3  Sat Oct 29 18:38:00 +0000 2022  1586427143302443008            None   
4  Sat Oct 29 18:38:43 +0000 2022  1586427325264310273            None   

     user_name  followers_count  verified_user    user_location  \
0   TitanUp Ty             2256          False             None   
1     SickYooo               88          False             None   
2       Saturn              883          False          America   
3  💙KOHAKU225💙              127          False  Baton Rouge, LA   
4       maoooo                6          False             None   

                      user_description  reply_count  retweet_count  \
0  #LGM #TITANS #GBO🍊 #FJB #NJ #USA 🇺🇸            5             51   
1                                   ..            3            168   
2       #براندازم #نه_به_جمهوری_اسلامی            0              7   
3                                 None            3             16   
4                                 None            2             10   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [34]:
#Taking the null count to check for any discrepancies
df_nuls_cnt = df_coords_analysis_df.select([
                F.count(
                    F.when(df_coords_analysis_df[column].isNull(), column)
                ).alias(column) for column in df_coords_analysis_df.columns
            ]).toPandas()

# display.max_columns : int

In [36]:
df_nuls_cnt

,created_at,id,geo_coordinates,user_name,followers_count,verified_user,user_location,user_description,reply_count,retweet_count,retweeted_status,tweet_text,text
0,0,0,997584,0,0,0,451915,231810,352608,352608,352608,0,0


In [41]:
%%time 
df_coords_analysis_df.write.mode("overwrite").parquet("gs://msca-bdp-students-bucket/shared_data/kishorkumarreddy/df_coords_analysis_df/")

CPU times: user 240 ms, sys: 30.4 ms, total: 270 ms
Wall time: 3min 46s
